In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

import os
import numpy as np
import pandas as pd
# Custom utils
from utils.simulator.simulator import MCSimulation
# Tf imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

2023-03-22 02:07:51.009748: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
def custom_derivative_model(x, model, multioutput = True, mode = 'centered'):
    h = 1e-1
    # Size x
    x_dim, y_dim, _ = x.shape
    # Gradient vector
    if multioutput:
        gradient = np.zeros((x_dim, y_dim, 1))
    else:
        gradient = np.zeros((x_dim, y_dim))
    for i in range(y_dim):
        # Vector for partial derivative estimation
        offset_tensor = np.zeros((x_dim, y_dim, 1))
        offset_tensor[:, i] = h
        offset_tensor = tf.convert_to_tensor(offset_tensor,
                                             dtype = tf.float32)
        # Constantes:
        denominator = h
        if mode == 'progressive':
            numerator = model(
                tf.math.add(x, offset_tensor)
            ) - model(
                x
            ) 
        elif mode == 'regressive':
            numerator = model(
                x
            ) - model(
                tf.math.subtract(x, offset_tensor)
            )
        elif mode == 'centered':
            numerator = tf.math.subtract(
                model(
                    tf.math.add(x, offset_tensor)
                ), model(
                    tf.math.subtract(x, offset_tensor)
                )
            )
        denominator = 2 * h
        print(f'Numerator: {numerator}')
        print(f'Denominator: {denominator}')
        gradient [:, i, :] = numerator / denominator
    gradient = tf.convert_to_tensor(gradient,
                                        dtype = tf.float32)
    return gradient

In [13]:
input_layer = keras.Input(shape = (10, 1), name='input_nn')
output_layer = layers.GRU(1, 
                dropout = 0.0,
                input_shape = (10, 1),
                return_sequences = True,
                name = 'sequential_layer')(input_layer)
custom_model = keras.Model(
    inputs=[input_layer],
    outputs=[output_layer],
    name = 'test_model'
)

In [14]:
x = tf.convert_to_tensor(
    np.array(range(0,30)).reshape((3,10, 1)),
    dtype = tf.float32    
)
# Test 
import time
tf_start = time.time()
xs = tf.Variable(x, trainable = True, name = 'x')
with tf.GradientTape() as tape, tf.GradientTape() as tape_2:
    tape.watch(xs)
    tape_2.watch(xs)
    y = custom_model(xs)
tf_end = time.time()
print(f'Execution time: {tf_end - tf_start}')
# This represents dV/dX
grads = tape.gradient(y, {
    'x':xs
})
jacobian = tape_2.jacobian(y, {
    'x':xs
})

Execution time: 0.032103776931762695


In [17]:
grads

{'x': <tf.Tensor: shape=(3, 10, 1), dtype=float32, numpy=
 array([[[ 3.18759531e-01],
         [ 2.56062269e-01],
         [ 1.87381893e-01],
         [ 1.20098025e-01],
         [ 6.42444417e-02],
         [ 2.52150819e-02],
         [ 2.90626287e-03],
         [-6.33487664e-03],
         [-7.30305910e-03],
         [-4.22189571e-03]],
 
        [[-1.55025691e-01],
         [-1.14448510e-01],
         [-8.25277418e-02],
         [-5.80778755e-02],
         [-3.97756360e-02],
         [-2.63540596e-02],
         [-1.67015865e-02],
         [-9.89617221e-03],
         [-5.20195952e-03],
         [-2.04787776e-03]],
 
        [[-1.86392535e-02],
         [-1.32019781e-02],
         [-9.22937598e-03],
         [-6.34808186e-03],
         [-4.27555619e-03],
         [-2.79885833e-03],
         [-1.75850012e-03],
         [-1.03559194e-03],
         [-5.42031135e-04],
         [-2.12762054e-04]]], dtype=float32)>}

In [21]:
custom_start = time.time()
custom_grads = custom_derivative_model(x, custom_model)
custom_end = time.time()
print(f'Execution time custom: {custom_end - custom_start}')

Numerator: [[[ 0.021808  ]
  [ 0.01288341]
  [ 0.00839078]
  [ 0.00581023]
  [ 0.00420725]
  [ 0.00316256]
  [ 0.00246066]
  [ 0.00197893]
  [ 0.00164258]
  [ 0.00140393]]

 [[-0.00335634]
  [-0.00327525]
  [-0.00320315]
  [-0.00314112]
  [-0.00308898]
  [-0.00304592]
  [-0.00301072]
  [-0.00298235]
  [-0.00295955]
  [-0.00294137]]

 [[-0.00037433]
  [-0.00037374]
  [-0.00037333]
  [-0.00037301]
  [-0.0003727 ]
  [-0.00037251]
  [-0.00037234]
  [-0.00037219]
  [-0.00037205]
  [-0.00037195]]]
Denominator: 0.2


ValueError: could not broadcast input array from shape (3,10,1) into shape (3,1)